In [1]:
import hopsworks
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
from thesis.fully_synthetic_data.src.users import generate_users
from thesis.fully_synthetic_data.src.events import generate_events 
from thesis.fully_synthetic_data.src.interactions import generate_interactions 


## <span style='color:#ff5f27'> ⚙️ Data Generation</span>


---

### <span style='color:#ff5f27'> 👥 Users Data Generation</span>

In [21]:
# Login to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()


2025-05-27 18:24:45,364 INFO: Initializing external client
2025-05-27 18:24:45,366 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 18:24:46,788 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [2]:
# clean text columns in the generated data
def clean_text(text):
    if not isinstance(text, str):
        return "unknown"
    text = text.lower().strip()
    text = text.replace(',', ' ')
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)     # collapse multiple spaces
    return text if text else "unknown"

def clean_text_columns(df):
    if 'title' in df.columns:
        df["title"] = df["title"].apply(lambda x: clean_text(x) if isinstance(x, str) and x.strip() else "unknown")
    if 'user_interests' in df.columns:
        df["user_interests"] = df["user_interests"].apply(
            lambda x: clean_text(x) if isinstance(x, str) and x.strip() else "unknown")
        
    return df



In [3]:
# Generate user data
print("Generating user data...")
num_users = 50000
users = generate_users(num_users)
users_df = pd.DataFrame(users)

# Clean the dataframes
users_df = clean_text_columns(users_df)
print("Done!")

Generating user data...
Done!


In [4]:
# generate event data
print("Generating event data...")
num_events = 10000
events = generate_events(num_events)
events_df = pd.DataFrame(events)

# Clean the dataframes
events_df = clean_text_columns(events_df)
print("Done!")

Generating event data...
Done!


In [11]:
events_df.head()

,event_id,title,category,event_lat,event_lon,event_city,start_time,duration,weather_condition,temperature,attendance_rate,event_indoor_capability
0,FC369F,customerfocused leadingedge protocol sports fi...,Sports & Fitness,35.637997,140.002170,Tokyo,2025-06-25 13:17:55.718852,480,Cloudy,21.2,16.495123,False
1,CK086Y,visionary asymmetric access immersive experien...,Immersive Experiences,40.647622,-73.792194,New York,2025-06-16 13:13:06.621199,180,Clear,19.0,48.351475,True
2,ZJ670K,usercentric hybrid timeframe business networki...,Business & Networking,48.559383,2.530325,Paris,2025-06-27 09:00:26.803485,360,Snow,11.6,43.398907,True
3,WP521V,standalone fullrange application food drink in...,Food & Drink,19.445576,72.773147,Mumbai,2025-07-28 19:40:29.896955,180,Windy,28.4,22.903765,False
4,AO573V,devolved contentbased hierarchy food drink in ...,Food & Drink,40.540794,-73.761227,New York,2025-07-26 18:25:27.929875,480,Rain,14.5,4.538231,False


In [ ]:
# generate interactions data
print("Generating interaction data...")
num_interactions = 500000
interactions = generate_interactions(users_df, events_df, num_interactions)
interactions_df = pd.DataFrame(interactions)

# Add interaction_label for model training
interactions_df['interaction_label'] = interactions_df['interaction_type'].apply(
    lambda x: 1 if x in ['yes', 'invited & yes', 'maybe', 'invited & maybe'] else 0
)
# Clean the dataframes
interactions_df = clean_text_columns(interactions_df)
print("Done!")


In [15]:
# Save all generated data to csv
events_df.to_csv("/home/nkama/masters_thesis_project/thesis/fully_synthetic_data/data/events.csv",index=False )
users_df.to_csv("/home/nkama/masters_thesis_project/thesis/fully_synthetic_data/data/users.csv",index=False)
interactions_df.to_csv("/home/nkama/masters_thesis_project/thesis/fully_synthetic_data/data/interactions.csv",index=False)

In [4]:
# import pandas as pd
# events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/events" )
# users_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/users.csv")
# interactions_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/interactions.csv")

In [14]:
# rename similar columns names found in both users and events data
users_df.rename(columns={
    "weather_condition" : "user_weather_condition",
    "temperature": "user_temperature",
    "precipitation":"user_precipitation"
}, inplace = True)

In [15]:
users_df.columns

Index(['user_id', 'user_lat', 'user_lon', 'user_city',
       'indoor_outdoor_preference', 'age', 'user_interests', 'signup_date',
       'social_connectedness', 'user_weather_condition', 'user_temperature',
       'user_precipitation'],
      dtype='object')

In [22]:
# check length of all generated datasets
len(events_df), len(users_df), len(interactions_df)

(10000, 50000, 500000)

In [25]:
interactions_df.head()


,interaction_id,user_id,event_id,interaction_type,interaction_time,interaction_distance_to_event,interaction_label
0,GM322S,DK924U,SW955O,invited & no,2025-07-22 02:34:22.544292,15.442082,0
1,IP622F,PQ743E,KK727U,invited & no,2025-08-26 14:43:31.919623,18.905958,0
2,QR516Z,NX067Y,KV778X,invited & maybe,2025-10-12 20:59:39.916672,58.743016,1
3,KM455K,AJ787D,AF604G,yes,2025-08-19 08:32:51.533099,22.280995,1
4,NC809A,TA859C,DB908U,yes,2025-07-18 21:26:23.948365,56.605419,1


In [28]:
#rename event category column to match event_type in semi-synthetic data
events_df.rename(columns={"category":"event_type"},inplace = True)

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

### Create feature groups

In [30]:
# Create user feature groups
print("Creating feature groups...")
users_fg = fs.get_or_create_feature_group(
    name="users",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
    description="User features for weather-based event recommendation"
)

users_fg.insert(users_df) # Insert data into feature groups
print('Done ✅')

Creating feature groups...


Uploading Dataframe: 100.00% |██████████| Rows 50000/50000 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: users_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/users_1_offline_fg_materialization/executions
Done ✅


In [37]:
# Create events feature groups
events_fg = fs.get_or_create_feature_group(
    name="events",
    version=1,
    primary_key=["event_id"],
    online_enabled=True,
    description="Event features for weather-based event recommendation"
)

events_fg.insert(events_df) # Insert data into feature groups
print('Done ✅')


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fg/1479135


Uploading Dataframe: 100.00% |██████████| Rows 10000/10000 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: events_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/events_1_offline_fg_materialization/executions
Done ✅


In [31]:
# Create interactions feature groups
interactions_fg = fs.get_or_create_feature_group(
    name="interactions",
    version=1,
    primary_key=["interaction_id","user_id", "event_id"],
    online_enabled=True,
    description="User-event interactions for weather-based event recommendation"
)

interactions_fg.insert(interactions_df) # Insert data into feature groups

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fg/1477094


Uploading Dataframe: 100.00% |██████████| Rows 500000/500000 | Elapsed Time: 00:43 | Remaining Time: 00:00


Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/interactions_1_offline_fg_materialization/executions


(Job('interactions_1_offline_fg_materialization', 'SPARK'), None)

### create ranking feature group

In [32]:
#create ranking feature group
events_interactions_df = pd.merge(
    interactions_df, 
    events_df, 
    on='event_id', 
    how='inner'  
)

ranking_df = pd.merge(
    events_interactions_df, 
    users_df, 
    on='user_id', 
    how='inner',
    #suffixes=('', '_event') 
)

### create weather ranking data with weather information

In [33]:
ranking_df_with_weather = ranking_df.drop(
    ['interaction_id', 'interaction_type','interaction_time',\
     'start_time', 'signup_date','social_connectedness'], 
    axis=1
)

ranking_fg_weather = fs.get_or_create_feature_group(
    name="weather_ranking",
    description="Ranking Data with weather data.",
    version=1,
    primary_key=["user_id", "event_id"],
    online_enabled=True,
    #features=ranking_df_with_weather.columns.to_list()  # ← critical!
)

ranking_fg_weather.insert(ranking_df_with_weather)
print('Done ✅')

#save to local disk
ranking_df_with_weather.to_csv("/home/nkama/masters_thesis_project/thesis/fully_synthetic_data/data/ranking_df_with_weather2.csv")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fg/1479133


Uploading Dataframe: 100.00% |██████████| Rows 501500/501500 | Elapsed Time: 01:20 | Remaining Time: 00:00


Launching job: weather_ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/weather_ranking_1_offline_fg_materialization/executions
Done ✅


### create ranking data without weather information

In [36]:
# create ranking data without weather information
ranking_df_without_weather = ranking_df.drop(['interaction_id', 'interaction_type',
       'interaction_time', 'start_time','weather_condition',
       'temperature', 'indoor_outdoor_preference',
       'signup_date', 'social_connectedness'],axis=1)

ranking_fg_without_weather = fs.get_or_create_feature_group(
    name="no_weather_ranking",
    description="Ranking Data without weather conditions.",
    version=1,
    primary_key=["user_id", "event_id"],
    online_enabled=True,
    #features=ranking_df_without_weather.columns.to_list()
)

ranking_fg_without_weather.insert(ranking_df_without_weather)
print('Done ✅')

#save to local disk
ranking_df_without_weather.to_csv("/home/nkama/masters_thesis_project/thesis/fully_synthetic_data/data/ranking_df_without_weather2.csv")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fg/1479134


Uploading Dataframe: 100.00% |██████████| Rows 501500/501500 | Elapsed Time: 01:13 | Remaining Time: 00:00


Launching job: no_weather_ranking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220788/jobs/named/no_weather_ranking_1_offline_fg_materialization/executions
Done ✅
